In [48]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.style.use('ggplot')


%store -r WORKDIR

if 'WORKDIR' not in dir():
    WORKDIR = 'C:/Users/thewr/git/mit_data_science.git/'


data_raw_file = WORKDIR + '/Data/Raw/energy_consumption_data_modeling.parquet'
data_proc_file = WORKDIR + '/Data/Processed/energy_consumption_data_modeling.parquet'

dataset_max_date  = '2008-06-02'
dataset_min_date = '2006-01-01'

#dataset_max_date  = '2006-12-30'
#dataset_min_date = '2006-01-01'


pd.plotting.register_matplotlib_converters()

# Leitura dos Dados 

In [49]:
pjme = pd.read_parquet(data_raw_file)

print('shape:', pjme.shape)
print('columns:', pjme.columns)
pjme.head()

shape: (145366, 2)
columns: Index(['Datetime', 'PJME_MW'], dtype='object')


Datetime  PJME_MW
0  2002-01-01 01:00:00    30393
1  2002-01-01 02:00:00    29265
2  2002-01-01 03:00:00    28357
3  2002-01-01 04:00:00    27899
4  2002-01-01 05:00:00    28057

In [50]:
pjme = pjme.set_index('Datetime')

In [51]:
pjme.head()

PJME_MW
Datetime                    
2002-01-01 01:00:00    30393
2002-01-01 02:00:00    29265
2002-01-01 03:00:00    28357
2002-01-01 04:00:00    27899
2002-01-01 05:00:00    28057

In [52]:
pjme = pjme[(pjme.index > dataset_min_date) & (pjme.index < dataset_max_date)]

In [53]:
pjme.head()

PJME_MW
Datetime                    
2006-01-01 00:00:00    30293
2006-01-01 01:00:00    28884
2006-01-01 02:00:00    27556
2006-01-01 03:00:00    26484
2006-01-01 04:00:00    25822

In [54]:
pjme.tail()

PJME_MW
Datetime                    
2008-06-01 19:00:00    33656
2008-06-01 20:00:00    33199
2008-06-01 21:00:00    33665
2008-06-01 22:00:00    33877
2008-06-01 23:00:00    31439

In [55]:
pjme = pjme.reset_index()

In [56]:
pjme['Datetime'] = pjme['Datetime'].astype('datetime64[ns]')

In [57]:
pjme_duplicados = pjme[pjme.duplicated(subset=['Datetime'])]

In [58]:
pjme_duplicados.count()

Datetime    0
PJME_MW     0
dtype: int64

In [59]:
pjme = pjme.drop_duplicates(subset=['Datetime'], keep='first')

In [60]:
pjme_duplicados = pjme[pjme.duplicated(subset=['Datetime'])]

In [61]:
pjme_duplicados.count()


Datetime    0
PJME_MW     0
dtype: int64

# Estatistica dos dados 

In [62]:
pjme.describe()

PJME_MW
count  21187.000000
mean   32755.027234
std     6317.268194
min    19690.000000
25%    28417.000000
50%    32339.000000
75%    35963.000000
max    62009.000000

In [63]:
pjme.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21187 entries, 0 to 21186
Data columns (total 2 columns):
Datetime    21187 non-null datetime64[ns]
PJME_MW     21187 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 496.6 KB


# Correção do tipo dos dados

In [64]:
pjme['Datetime'] = pjme['Datetime'].astype('datetime64[ns]')

data_types = {
    'PJME_MW': float    
}
for cname, dtype in data_types.items():
    if dtype == int:
        pjme.loc[data[cname].astype(str) == '',cname] = '0'
    
    pjme[cname] = pjme[cname].astype(dtype)
    
pjme.describe()

PJME_MW
count  21187.000000
mean   32755.027234
std     6317.268194
min    19690.000000
25%    28417.000000
50%    32339.000000
75%    35963.000000
max    62009.000000

In [65]:
import random
zones = ["pacific", "mountain", "central", "eastern"]
zone = random.choice(zones)
print(zone)
    

mountain


In [66]:
pjme.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21187 entries, 0 to 21186
Data columns (total 2 columns):
Datetime    21187 non-null datetime64[ns]
PJME_MW     21187 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 496.6 KB


In [67]:
pjme.tail()

Datetime  PJME_MW
21182 2008-06-01 19:00:00  33656.0
21183 2008-06-01 20:00:00  33199.0
21184 2008-06-01 21:00:00  33665.0
21185 2008-06-01 22:00:00  33877.0
21186 2008-06-01 23:00:00  31439.0

# Verificar Valores Nulos

In [68]:
pjme.isnull().sum()

Datetime    0
PJME_MW     0
dtype: int64

# Exportar Base de Dados 

In [69]:
pjme.to_parquet(data_proc_file)